In [1]:
import numpy as np

from pw_scf import plane_wave_basis
from pw_scf import get_nonlocal_pseudopotential_matrix_elements
from pw_scf import get_local_pseudopotential_gth
from pw_scf import get_matrix_elements

from pyscf.pbc import gto

In [2]:
def print_gth_parameters(cell, basis):

    """

    print the GTH pseudopotential parameters

    :param cell: the unit cell
    :param basis: planewave basis information (contains gth parameters)

    """

    print('')
    print('    ==> GTH pseudopotential parameters (local) <==')
    print('')


    natom = len(cell._atom)

    for center in range (0, natom):
        
        params = basis.gth_params[center]

        print('        atom: %20s' % ( cell._atom[center][0] ) )
        print('        Zion: %20i' % ( params.Zion ) )
        print('        rloc: %20.12lf' % ( params.rloc ) )
        print('        c1:   %20.12lf' % ( params.local_cn[0] ) )
        print('        c2:   %20.12lf' % ( params.local_cn[1] ) )
        print('        c3:   %20.12lf' % ( params.local_cn[2] ) )
        print('        c4:   %20.12lf' % ( params.local_cn[3] ) )
        print('')

    print('    ==> GTH pseudopotential parameters (non-local) <==')
    print('')

    for center in range (0, natom):
        
        params = basis.gth_params[center]

        print('        atom: %20s' % ( cell._atom[center][0] ) )
        rl = params.rl
        print('        rl:   %20.12lf %20.12lf %20.12lf' % ( rl[0], rl[1], rl[2] ) )
        print('        h^l_ij:')

        for l in range (0, 3):
            h = params.hgth
            print('')
            print('            l = %i' % ( l ) )
            for i in range (0, 3):
                print('            %20.12lf %20.12lf %20.12lf' % ( h[l, i, 0], h[l, i, 1], h[l, i, 2] ) )
        print('')

In [3]:
# build a unit cell
a = np.eye(3) * 4.0
atom = 'Ag 0 0 0'

cell = gto.M(a = a,
             atom = atom,
             unit = 'bohr',
             basis = 'sto-3g',
             pseudo = 'gth-blyp',
             precision = 1.0e-8,
             spin = 1,
             dimension = 3)
cell.build()

In [4]:
# get plane wave basis information
basis = plane_wave_basis(cell,                            
                         ke_cutoff = 500.0 / 27.21138602,
                         n_kpts = [1, 1, 1])
print("")
print('    no. k-points:                                %20i' % ( len(basis.kpts) ) )
print('    KE cutoff (eV)                               %20.2f' % ( basis.ke_cutoff * 27.21138602 ) )
print('    no. basis functions (density):               %20i' % ( len(basis.g) ) )
print('    no. basis functions (orbitals, gamma point): %20i' % ( basis.n_plane_waves_per_k[0] ) )


    no. k-points:                                                   1
    KE cutoff (eV)                                             500.00
    no. basis functions (density):                               1935
    no. basis functions (orbitals, gamma point):                  251


In [5]:
# print gth pseudopotential parameters
print_gth_parameters(cell, basis)


    ==> GTH pseudopotential parameters (local) <==

        atom:                   Ag
        Zion:                   11
        rloc:       0.570000000000
        c1:         0.273298710000
        c2:         0.000000000000
        c3:         0.000000000000
        c4:         0.000000000000

    ==> GTH pseudopotential parameters (non-local) <==

        atom:                   Ag
        rl:         0.531136910000       0.625721230000       0.408327850000
        h^l_ij:

            l = 0
                  9.454478590000      -4.739864100000       0.595127480000
                 -4.739864100000       7.087113170000      -1.536612560000
                  0.595127480000      -1.536612560000       1.219648410000

            l = 1
                  3.906195330000      -1.717749750000       0.000000000000
                 -1.717749750000       2.032468910000       0.000000000000
                  0.000000000000       0.000000000000       0.000000000000

            l = 2
          

In [6]:
# grab the gth pseudopotential parameters (local):
# Zion, rloc, c1, c2, c3, c4

natom = len(cell._atom)

for center in range (0, natom):
    
    params = basis.gth_params[center]
    
    Zion = params.Zion
    rloc = params.rloc
    c1 = params.local_cn[0]
    c2 = params.local_cn[1]
    c3 = params.local_cn[2]
    c4 = params.local_cn[3]

In [7]:
# grab the gth pseudopotential parameters (nonlocal):
# rl, hlij
for center in range (0, natom):
    
    params = basis.gth_params[center]

    rl = params.rl
    h_lij = params.hgth
     

In [8]:
# get local component of the pseudopotential in 
# the planewave basis for the density (E <= 2 x ke_cutoff)
# vl(K) = Eq. 50, 51 of the note
vl = get_local_pseudopotential_gth(basis)

# get the matrix elements of vl: <G'|vl|G''> = vl(G'-G'')
# in the planewave basis for the orbitals (E <= ke_cutoff)
# note that the dimension of this basis may not be the 
# same for all k-points
for kid in range (0, len(basis.kpts) ):
    vl_mat = get_matrix_elements(basis, kid, vl) 

In [9]:
# get nonlocal components of the pseudopotential matrix in
# the planewave basis for the oribtals (E <= ke_cutoff). note
# that the dimension of this basis may not be the same for
# all k-points. note also that we can replace the spherical
# harmonics in Eq. 59 of the note with their sum rule with 
# use_legendre = True
for kid in range (0, len(basis.kpts) ):
    
    # vnl(G',G'') = Eq. 59 of the note
    vnl_mat = get_nonlocal_pseudopotential_matrix_elements(basis, kid, use_legendre = False)
    
    # vnl(G',G'') = Eq. 59 of the note, with sum rule for the spherical harmonics
    vnl_mat_legendre = get_nonlocal_pseudopotential_matrix_elements(basis, kid, use_legendre = True)
    
    # check equivalence of vnl_mat and vnl_mat_legendre
    print(np.linalg.norm(vnl_mat - vnl_mat_legendre))

2.2995784460659235e-15
